# Generate example static catalog for S1 RTC on AWS

In [ ]:
import s3fs
import stactools
import pystac
from pystac import Catalog, Collection, CatalogType
from stactools.sentinel1.stac import create_item


%load_ext autoreload
%autoreload 2

In [ ]:
# install development version of stactools (after git clone https://github.com/scottyhq/sentinel1-rtc-stac.git )
#!pip install --no-deps -e ../stactools

## A sampling of items around CONUS and over time 

| Location | MGRS |
| - | - | 
| West coast (WA state) | 10UCU |
| US-canada border | 10UFV |
| Central UTM boundary 12/13 (Colorado) | 12SYJ, 13SBD | 
| Gulf coast | 17RMJ, 15RUN |
| East coast | 19TCH |

In [ ]:
s3 = s3fs.S3FileSystem(anon=True)

In [ ]:
# Get URIs for 3 tiles per year for each MGRS square
base_url = 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW'
grid_squares = ['10UCU', '10UFV', '12SYJ', '13SBD', '17RMJ', '15RUN', '19TCH']
years = [2016, 2017, 2018, 2019, 2020, 2021]
n_items = 3

paths = []
for mgrs in grid_squares:
    for year in years:
        utm_zone = mgrs[:2] 
        latitude_band = mgrs[2] 
        grid_square = mgrs[3:] 
        s3Path = f'{base_url}/{utm_zone}/{latitude_band}/{grid_square}/{year}'
        items = s3.ls(s3Path)
        if len(items)>0: #some tiles do not have data from 2016 or earlier
            paths += items[:n_items]

In [ ]:
len(paths)

In [ ]:
#write these out to a text file
with open('paths.txt','w') as f:
    f.write('\n'.join(paths))

In [ ]:
def s3_to_http(s3path, region='us-west-2'):
    s3prefix = 'sentinel-s1-rtc-indigo'
    newprefix = f'https://sentinel-s1-rtc-indigo.s3.{region}.amazonaws.com'
    http = s3path.replace(s3prefix, newprefix)
    #print(http)
    return http

In [ ]:
# Test creating single item
print(paths[0])
s3_to_http(f'{paths[0]}')

In [ ]:
import rasterio
with rasterio.open(s3_to_http(f'{paths[0]}')+'/local_incident_angle.tif') as src:
    print(src.profile)

In [ ]:
%%time
item = create_item(s3_to_http(f'{paths[0]}'))

In [ ]:
# View the JSON
#import json
#print(json.dumps(item.to_dict(), indent=1))

# Validate in-memory STAC
item.validate()

# Validate on-disk STAC
# pystac.read_file(f'{dst}/{dst}.json').validate()

In [ ]:
%%time 
# Generate all the items (could easily parallelize)
items = [create_item(s3_to_http(f'{i}')) for i in paths]

In [ ]:
# Add items to a catalog
catalog = Catalog(id='sentinel1-rtc',
                  description='Sentinel1 radiometric terrain corrected backscatter (RTC)'
                 )
catalog.add_items(items)
#print(len(catalog)) #TypeError: object of type 'Catalog' has no len()
catalog.describe()

In [ ]:
# Save the catalog in current directory
catalog.normalize_hrefs('./')
catalog.validate()
catalog.save(catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED)